# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f53bd5ef940>
├── 'a' --> tensor([[-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778]])
└── 'x' --> <FastTreeValue 0x7f53bd5ef250>
    └── 'c' --> tensor([[ 1.0127, -1.1618,  0.2166, -0.7350],
                        [ 1.1369,  0.8220,  0.6619, -0.2425],
                        [-0.3944, -0.7889, -1.1785, -1.9596]])

In [4]:
t.a

tensor([[-1.5258,  0.6921,  0.0682],
        [-0.4124,  0.2683, -1.9778]])

In [5]:
%timeit t.a

77.9 ns ± 0.342 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f53bd5ef940>
├── 'a' --> tensor([[ 0.0844,  0.7522, -1.1670],
│                   [ 0.8520,  0.3904, -1.0036]])
└── 'x' --> <FastTreeValue 0x7f53bd5ef250>
    └── 'c' --> tensor([[ 1.0127, -1.1618,  0.2166, -0.7350],
                        [ 1.1369,  0.8220,  0.6619, -0.2425],
                        [-0.3944, -0.7889, -1.1785, -1.9596]])

In [7]:
%timeit t.a = new_value

81.5 ns ± 1.96 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778]]),
    x: Batch(
           c: tensor([[ 1.0127, -1.1618,  0.2166, -0.7350],
                      [ 1.1369,  0.8220,  0.6619, -0.2425],
                      [-0.3944, -0.7889, -1.1785, -1.9596]]),
       ),
)

In [10]:
b.a

tensor([[-1.5258,  0.6921,  0.0682],
        [-0.4124,  0.2683, -1.9778]])

In [11]:
%timeit b.a

68.8 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2507,  0.6964, -0.6188],
               [ 0.4230, -0.5236, -0.9414]]),
    x: Batch(
           c: tensor([[ 1.0127, -1.1618,  0.2166, -0.7350],
                      [ 1.1369,  0.8220,  0.6619, -0.2425],
                      [-0.3944, -0.7889, -1.1785, -1.9596]]),
       ),
)

In [13]:
%timeit b.a = new_value

620 ns ± 23.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

970 ns ± 4.62 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 532 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

169 µs ± 723 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 706 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f53b8565f10>
├── 'a' --> tensor([[[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]],
│           
│                   [[-1.5258,  0.6921,  0.0682],
│                    [-0.4124,  0.2683, -1.9778]]])
└── 'x' --> <FastTreeValue 0x7f5314e70430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 49.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f53bd5ef550>
├── 'a' --> tensor([[-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778],
│                   [-1.5258,  0.6921,  0.0682],
│                   [-0.4124,  0.2683, -1.9778]])
└── 'x' --> <FastTreeValue 0x7f530e220580>
    └── 'c' --> tensor([[ 1.0127, -1.1618,  0.2166, -0.7350],
                        [ 1.1369,  0.8220,  0.6619, -0.2425],
                 

In [23]:
%timeit t_cat(trees)

35.8 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.8 µs ± 292 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]],
       
               [[-1.5258,  0.6921,  0.0682],
                [-0.4124,  0.2683, -1.9778]]]),
    x: Batch(
           c: tensor([[[ 1.0127, -1.1618,  0.2166, -0.7350],
                       [ 1.1369,  0.8220,  0.6619, -0.2425],
                       [-0.3944, -0.7889, -1.1785, -1.9596]],
         

In [26]:
%timeit Batch.stack(batches)

129 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778],
               [-1.5258,  0.6921,  0.0682],
               [-0.4124,  0.2683, -1.9778]]),
    x: Batch(
           c: tensor([[ 1.0127, -1.1618,  0.2166, -0.7350],
                      [ 1.1369,  0.8220,  0.6619, -0.2425],
                      [-0.3944, -0.7889, -1.1785, -1.9596],
                      [ 1.0127, -1.1618,  0.2166, -0.7350],
                      [ 1.1369,  

In [28]:
%timeit Batch.cat(batches)

363 µs ± 7.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

436 µs ± 29.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
